<a href="https://colab.research.google.com/github/rizalramle21/rice-leaf-disease-classification-using-SVM/blob/main/svmGLCMfitur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Library yang dibutuhkan
import pandas as pd #manipulasi data dan analisis data seperti excel
import numpy as np #operasi numerik dan array
import cv2 #Membaca, memanipulasi, dan menganalisis gambar.
import os #Mengimpor gambar, berinteraksi dengan sistem file.
import csv #Mengelola data dalam format CSV.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Leaf Blight

In [ ]:
from skimage.feature import graycomatrix, graycoprops

# Path ke folder gambar
path = '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Bacterial Blight'
data = os.listdir(path)

# Inisialisasi list fitur
features = []
labels = []

# Threshold HSV daun
lower = np.array([59, 0, 180])
upper = np.array([135, 114, 255])

for image_name in data:
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Gagal membuka: {image_name}")
        continue

    # Resize ke 256x256
    image = cv2.resize(image, (256, 256))

    # --- Ekstraksi Fitur Tekstur (GLCM) ---
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- Ekstraksi Fitur Warna HSV ---
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    masked_hsv = cv2.bitwise_and(hsv, hsv, mask=mask)

    h, s, v = cv2.split(masked_hsv)
    h_mean = np.mean(h[mask > 0])
    s_mean = np.mean(s[mask > 0])
    v_mean = np.mean(v[mask > 0])

    # Gabungkan fitur
    feature_row = [contrast, homogeneity, energy, correlation, h_mean, s_mean, v_mean]
    features.append(feature_row)
    labels.append(1)  # Ganti sesuai label kategori jika multiclass

# Simpan ke DataFrame
columns = ['Contrast', 'Homogeneity', 'Energy', 'Correlation', 'Hue_Mean', 'Saturation_Mean', 'Value_Mean']
feature_df = pd.DataFrame(features, columns=columns)
label_df = pd.DataFrame(labels, columns=['Label'])
final_df = pd.concat([feature_df, label_df], axis=1)

# Lihat hasil
print(final_df.head())


     Contrast  Homogeneity    Energy  Correlation   Hue_Mean  Saturation_Mean  \
0   84.436940     0.400464  0.108820     0.983577  30.597691       185.982306   
1  154.303178     0.341230  0.082394     0.971853  33.202920       195.128820   
2  159.526348     0.306531  0.059587     0.974089  30.056350       180.344562   
3   93.359842     0.385140  0.110287     0.985092  31.922045       196.354772   
4  190.045879     0.337747  0.098726     0.963700  33.215077       200.112790   

   Value_Mean  Label  
0  132.878529      1  
1  111.088765      1  
2  131.705742      1  
3  134.908184      1  
4  108.260898      1  


In [ ]:
#INI BUAT DOWNLOAD FILE
from skimage.feature import graycomatrix, graycoprops
import cv2
import os
import numpy as np
import pandas as pd
import shutil
from google.colab import files

# ==== PERSIAPAN ====
# Folder input (gambar mentah)
input_path = '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Bacterial Blight'
image_list = os.listdir(input_path)

# Folder output untuk gambar hasil
output_folder = '/content/processed_images'
os.makedirs(output_folder, exist_ok=True)

# ==== INISIALISASI ====
features = []
labels = []

# HSV threshold untuk masking daun
lower = np.array([25, 40, 40])
upper = np.array([90, 255, 255])

# ==== LOOP GAMBAR ====
for image_name in image_list:
    image_path = os.path.join(input_path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"❌ Gagal membuka: {image_name}")
        continue

    # Resize ke 256x256
    image = cv2.resize(image, (256, 256))

    # --- GLCM Feature ---
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- HSV Feature ---
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    masked_hsv = cv2.bitwise_and(hsv, hsv, mask=mask)

    # Hitung Mean HSV hanya dari bagian objek
    h, s, v = cv2.split(masked_hsv)
    h_mean = np.mean(h[mask > 0])
    s_mean = np.mean(s[mask > 0])
    v_mean = np.mean(v[mask > 0])

    # Gabungkan fitur
    feature_row = [contrast, homogeneity, energy, correlation, h_mean, s_mean, v_mean]
    features.append(feature_row)

    # Ganti label sesuai kebutuhan (sementara 1 untuk semua)
    labels.append(1)

    # --- Simpan Gambar yang Sudah Dimasking ---
    masked_rgb = cv2.cvtColor(masked_hsv, cv2.COLOR_HSV2BGR)
    save_path = os.path.join(output_folder, f'processed_{image_name}')
    cv2.imwrite(save_path, masked_rgb)

# ==== SIMPAN FITUR KE CSV ====
columns = ['Contrast', 'Homogeneity', 'Energy', 'Correlation', 'Hue_Mean', 'Saturation_Mean', 'Value_Mean']
df_features = pd.DataFrame(features, columns=columns)
df_labels = pd.DataFrame(labels, columns=['Label'])
final_df = pd.concat([df_features, df_labels], axis=1)


# ==== ZIP GAMBAR HASIL ====
zip_path = '/content/processed_images.zip'
shutil.make_archive('/content/processed_images', 'zip', output_folder)
print("✅ ZIP gambar berhasil dibuat:", zip_path)

# ==== DOWNLOAD FILE ====
files.download(zip_path)


✅ ZIP gambar berhasil dibuat: /content/processed_images.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Leaf Smut

In [ ]:
from skimage.feature import graycomatrix, graycoprops
import cv2
import os
import numpy as np
import pandas as pd

# Path ke folder gambar
path = '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Leaf Smut'
data = os.listdir(path)

# Inisialisasi list fitur
features = []
labels = []

# Threshold HSV daun
lower = np.array([25, 40, 40])
upper = np.array([90, 255, 255])

for image_name in data:
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Gagal membuka: {image_name}")
        continue

    # Resize ke 256x256
    image = cv2.resize(image, (256, 256))

    # --- Ekstraksi Fitur Tekstur (GLCM) ---
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- Ekstraksi Fitur Warna HSV ---
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    masked_hsv = cv2.bitwise_and(hsv, hsv, mask=mask)

    h, s, v = cv2.split(masked_hsv)
    h_mean = np.mean(h[mask > 0])
    s_mean = np.mean(s[mask > 0])
    v_mean = np.mean(v[mask > 0])

    # Gabungkan fitur
    feature_row = [contrast, homogeneity, energy, correlation, h_mean, s_mean, v_mean]
    features.append(feature_row)
    labels.append(2)  # Ganti sesuai label kategori jika multiclass

# Simpan ke DataFrame
columns = ['Contrast', 'Homogeneity', 'Energy', 'Correlation', 'Hue_Mean', 'Saturation_Mean', 'Value_Mean']
feature_df = pd.DataFrame(features, columns=columns)
label_df = pd.DataFrame(labels, columns=['Label'])
final_df2 = pd.concat([feature_df, label_df], axis=1)

# Lihat hasil
print(final_df2())


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Leaf Smut'

## Brown Spot

In [ ]:
from skimage.feature import graycomatrix, graycoprops
import cv2
import os
import numpy as np
import pandas as pd

# Path ke folder gambar
path = '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Brown Spot'
data = os.listdir(path)

# Inisialisasi list fitur
features = []
labels = []

# Threshold HSV daun
lower = np.array([25, 40, 40])
upper = np.array([90, 255, 255])

for image_name in data:
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Gagal membuka: {image_name}")
        continue

    # Resize ke 256x256
    image = cv2.resize(image, (256, 256))

    # --- Ekstraksi Fitur Tekstur (GLCM) ---
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- Ekstraksi Fitur Warna HSV ---
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    masked_hsv = cv2.bitwise_and(hsv, hsv, mask=mask)

    h, s, v = cv2.split(masked_hsv)
    h_mean = np.mean(h[mask > 0])
    s_mean = np.mean(s[mask > 0])
    v_mean = np.mean(v[mask > 0])

    # Gabungkan fitur
    feature_row = [contrast, homogeneity, energy, correlation, h_mean, s_mean, v_mean]
    features.append(feature_row)
    labels.append(3)  # Ganti sesuai label kategori jika multiclass

# Simpan ke DataFrame
columns = ['Contrast', 'Homogeneity', 'Energy', 'Correlation', 'Hue_Mean', 'Saturation_Mean', 'Value_Mean']
feature_df = pd.DataFrame(features, columns=columns)
label_df = pd.DataFrame(labels, columns=['Label'])
final_df3 = pd.concat([feature_df, label_df], axis=1)

# Lihat hasil
print(final_df3.head())


     Contrast  Homogeneity    Energy  Correlation   Hue_Mean  Saturation_Mean  \
0  359.131754     0.451890  0.346033     0.975948  33.244686       199.898312   
1  814.301544     0.529998  0.479099     0.902438  35.831925       189.050169   
2  875.269018     0.416149  0.363752     0.923567  32.620898       198.904046   
3  490.481683     0.197207  0.108366     0.898299  32.236518       153.327442   
4  620.454152     0.231545  0.153459     0.904452  34.928747       142.722743   

   Value_Mean  Label  
0  128.925217      3  
1  173.179332      3  
2  140.900949      3  
3  193.255051      3  
4  163.398421      3  


##Hispa

In [ ]:
from skimage.feature import graycomatrix, graycoprops
import cv2
import os
import numpy as np
import pandas as pd

# Path ke folder gambar
path = '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Hispa'
data = os.listdir(path)

# Inisialisasi list fitur
features = []
labels = []

# Threshold HSV daun
lower = np.array([25, 40, 40])
upper = np.array([90, 255, 255])

for image_name in data:
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Gagal membuka: {image_name}")
        continue

    # Resize ke 256x256
    image = cv2.resize(image, (256, 256))

    # --- Ekstraksi Fitur Tekstur (GLCM) ---
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- Ekstraksi Fitur Warna HSV ---
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    masked_hsv = cv2.bitwise_and(hsv, hsv, mask=mask)

    h, s, v = cv2.split(masked_hsv)
    h_mean = np.mean(h[mask > 0])
    s_mean = np.mean(s[mask > 0])
    v_mean = np.mean(v[mask > 0])

    # Gabungkan fitur
    feature_row = [contrast, homogeneity, energy, correlation, h_mean, s_mean, v_mean]
    features.append(feature_row)
    labels.append(4)  # Ganti sesuai label kategori jika multiclass

# Simpan ke DataFrame
columns = ['Contrast', 'Homogeneity', 'Energy', 'Correlation', 'Hue_Mean', 'Saturation_Mean', 'Value_Mean']
feature_df = pd.DataFrame(features, columns=columns)
label_df = pd.DataFrame(labels, columns=['Label'])
final_df4 = pd.concat([feature_df, label_df], axis=1)

# Lihat hasil
print(final_df4.head())


     Contrast  Homogeneity    Energy  Correlation   Hue_Mean  Saturation_Mean  \
0  219.792829     0.203625  0.034181     0.894858  35.460446       215.784990   
1  137.877879     0.171638  0.052360     0.954013  35.015680       164.630560   
2  212.258404     0.200170  0.040696     0.936878  32.652117       144.345773   
3  650.246732     0.141143  0.039339     0.848535  37.035389       158.024200   
4  195.721894     0.384695  0.227419     0.948163  33.173952       159.771632   

   Value_Mean  Label  
0  100.992224      4  
1  106.690400      4  
2  113.009573      4  
3   93.893348      4  
4  119.229111      4  


## Blast


In [ ]:
from skimage.feature import graycomatrix, graycoprops
import cv2
import os
import numpy as np
import pandas as pd

# Path ke folder gambar
path = '/content/drive/MyDrive/Colab Notebooks/Dataset/dataset/Blast'
data = os.listdir(path)

# Inisialisasi list fitur
features = []
labels = []

# Threshold HSV daun
lower = np.array([25, 40, 40])
upper = np.array([90, 255, 255])

for image_name in data:
    image_path = os.path.join(path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Gagal membuka: {image_name}")
        continue

    # Resize ke 256x256
    image = cv2.resize(image, (256, 256))

    # --- Ekstraksi Fitur Tekstur (GLCM) ---
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    # --- Ekstraksi Fitur Warna HSV ---
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    masked_hsv = cv2.bitwise_and(hsv, hsv, mask=mask)

    h, s, v = cv2.split(masked_hsv)
    h_mean = np.mean(h[mask > 0])
    s_mean = np.mean(s[mask > 0])
    v_mean = np.mean(v[mask > 0])

    # Gabungkan fitur
    feature_row = [contrast, homogeneity, energy, correlation, h_mean, s_mean, v_mean]
    features.append(feature_row)
    labels.append(5)  # Ganti sesuai label kategori jika multiclass

# Simpan ke DataFrame
columns = ['Contrast', 'Homogeneity', 'Energy', 'Correlation', 'Hue_Mean', 'Saturation_Mean', 'Value_Mean']
feature_df = pd.DataFrame(features, columns=columns)
label_df = pd.DataFrame(labels, columns=['Label'])
final_df5 = pd.concat([feature_df, label_df], axis=1)

# Lihat hasil
print(final_df5.head())


     Contrast  Homogeneity    Energy  Correlation   Hue_Mean  Saturation_Mean  \
0  511.938870     0.709708  0.369471     0.829722  39.481703       161.971377   
1  399.549972     0.262667  0.044735     0.880770  38.124355       202.251290   
2  573.639084     0.327200  0.055583     0.741293  41.823836       206.282852   
3  527.649387     0.692749  0.221257     0.819052  44.424455       143.468260   
4  481.702394     0.579945  0.154301     0.837714  37.476834       156.318911   

   Value_Mean  Label  
0  140.313225      5  
1   88.264061      5  
2   93.769191      5  
3  107.546732      5  
4  115.702902      5  


In [ ]:
listdata = [final_df, final_df2, final_df3,final_df4, final_df5 ]
total = pd.concat(listdata, ignore_index=True)
total.to_csv("klasifikasi_daun.csv", index=False)
total.to_excel('klasifikasi_daun.xlsx', index=False)


In [ ]:
total.head()

,Contrast,Homogeneity,Energy,Correlation,Hue_Mean,Saturation_Mean,Value_Mean,Label
0,84.436940,0.400464,0.108820,0.983577,30.597691,185.982306,132.878529,1
1,154.303178,0.341230,0.082394,0.971853,33.202920,195.128820,111.088765,1
2,159.526348,0.306531,0.059587,0.974089,30.056350,180.344562,131.705742,1
3,93.359842,0.385140,0.110287,0.985092,31.922045,196.354772,134.908184,1
4,190.045879,0.337747,0.098726,0.963700,33.215077,200.112790,108.260898,1


In [ ]:
# prompt: download file langsung

from google.colab import files
total.to_csv('klasifikasi_daun.csv', encoding = 'utf-8-sig')
files.download('klasifikasi_daun.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>